In [1]:
import sys
import imaplib
import getpass
import email
import email.header
import datetime
import email.message
import csv

In [9]:
EMAIL_ACCOUNT = "ouedraogoaris@gmail.com"

# Use 'INBOX' to read inbox.  Note that whatever folder is specified, 
# after successfully running this script all emails in that folder 
# will be marked as read.
EMAIL_FOLDER = "Inbox"


def process_mailbox(M):


    date = (datetime.date.today() - datetime.timedelta(1)).strftime("%d-%b-%Y")
    rv, data = M.search(None, '(SENTSINCE {date})'.format(date=date))
    if rv != 'OK':
        print("No messages found!")
        return


    for num in data[0].split():
        rv, data = M.fetch(num, '(RFC822)')
        if rv != 'OK':
            print("ERROR getting message", num)
            return

        msg = email.message_from_bytes(data[0][1])
        hdr = email.header.make_header(email.header.decode_header(msg['Subject']))
        mes = email.message.EmailMessage(email.message.EmailMessage(msg['Message']))
        raw_email = data[0][1].decode("utf-8")
        email_message = email.message_from_string(raw_email)


        subject = str(hdr)
        print('Message %s: %s' % (num, subject))
        print('Raw Date:', msg['Date'])
        print(raw_email)
        temp_dict= {}
        temp_dict['Sender'] = email_message["From"]
        temp_dict['Subject'] = email_message["Subject"]
        temp_dict['Date'] = email_message["Date"]
        temp_dict['Body'] = email_message["raw_email"]
        temp_dict['Message-ID'] = email_message["Message-ID"]
        print(temp_dict)
        final_list.append(temp_dict)


        date_tuple = email.utils.parsedate_tz(msg['Date'])
        if date_tuple:
            local_date = datetime.datetime.fromtimestamp(
                email.utils.mktime_tz(date_tuple))
            print ("Local Date:", \
                local_date.strftime("%a, %d %b %Y %H:%M:%S"))



In [ ]:
M = imaplib.IMAP4_SSL('gmail.com')
#getpass.getpass()
try:
    rv, data = M.login(EMAIL_ACCOUNT,"288535122" )
except imaplib.IMAP4.error:
    print ("LOGIN FAILED!!! ")
    sys.exit(1)

print(rv, data)


final_list = [ ]

rv, data = M.select(EMAIL_FOLDER)
if rv == 'OK':
    print("Processing mailbox...\n")
    process_mailbox(M)
    M.close()
else:
    print("ERROR: Unable to open mailbox ", rv)

M.logout()

with open('test.csv', 'w', encoding='utf-8', newline = '') as csvfile: 
    fieldnames = ['Sender','Subject','Date','Body','Message-ID']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter = ',')
    writer.writeheader()
    for val in final_list:
        writer.writerow(val)